# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data 

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,5.97,35,75,2.57,US,1712014156
1,1,hawaiian paradise park,19.5933,-154.9731,25.73,82,100,5.66,US,1712014156
2,2,tocache,-8.1842,-76.5125,22.19,99,100,0.41,PE,1712014157
3,3,waitangi,-43.9535,-176.5597,14.34,80,100,1.34,NZ,1712014157
4,4,ilulissat,69.2167,-51.1000,-3.99,93,75,0.00,GL,1712014157


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points('Lng', 'Lat',
                                       geo=True,
                                       tiles='OSM',
                                       size = 'Humidity',
                                       frame_width = 650,
                                       frame_height = 500,
                                       color = 'City',
                                       alpha = 0.6
                                       )

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df[(city_data_df['Max Temp']>20) & (city_data_df['Max Temp']<30)]
ideal_df = ideal_temp_df[(ideal_temp_df['Cloudiness'] == 0) & (ideal_temp_df['Wind Speed'] < 4) & 
                         (ideal_temp_df['Humidity']>50)]
# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,le vauclin,14.5452,-60.8388,27.03,74,0,3.60,MQ,1712014191
115,115,fort bragg,35.1390,-79.0060,26.91,52,0,3.09,US,1712014196
133,133,martapura,-3.4167,114.8500,25.05,94,0,1.03,ID,1712014202
157,157,point vernon,-25.2500,152.8167,26.73,79,0,3.58,AU,1712014211
175,175,vallenar,-28.5708,-70.7581,22.55,51,0,1.08,CL,1712014218
176,176,kwang binh,17.4833,106.6000,27.11,60,0,2.68,VN,1712014218
203,203,salalah,17.0151,54.0924,26.05,53,0,1.54,OM,1712014228
231,231,al burayqah,30.4062,19.5739,21.60,51,0,3.40,LY,1712014238
324,324,dhahran,26.3032,50.1353,23.45,73,0,2.57,SA,1712014273
396,396,khon buri,14.5254,102.2459,26.41,73,0,2.47,TH,1712014303


### Step 3: Create a new DataFrame called `hotel_df`.

In [57]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = ideal_df[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,le vauclin,MQ,14.5452,-60.8388,74,
115,fort bragg,US,35.1390,-79.0060,52,
133,martapura,ID,-3.4167,114.8500,94,
157,point vernon,AU,-25.2500,152.8167,79,
175,vallenar,CL,-28.5708,-70.7581,51,
176,kwang binh,VN,17.4833,106.6000,60,
203,salalah,OM,17.0151,54.0924,53,
231,al burayqah,LY,30.4062,19.5739,51,
324,dhahran,SA,26.3032,50.1353,73,
396,khon buri,TH,14.5254,102.2459,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
print(hotel_df[hotel_df['City'] == 'khon buri']['Lat'])

396    14.5254
Name: Lat, dtype: float64


In [69]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
    'categories' : categories,
    'apiKey' : geoapify_key
}

 # get latitude, longitude from the DataFrame
lat = hotel_df[hotel_df['City'] == 'esperanza']['Lat']
lon = hotel_df[hotel_df['City'] == 'esperanza']['Lng']
    
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{lon},{lat},{radius}"
params["bias"] = f"bias:{lon},{lat}"
    
# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
    
# Convert the API response to JSON format
name_address = name_address.json()

print(name_address)

{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}


In [73]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
    'categories' : categories,
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
le vauclin - nearest hotel: Hôtel Macabou
fort bragg - nearest hotel: Airborne Inn Lodging
martapura - nearest hotel: Q-Hotel Grand Dafam
point vernon - nearest hotel: Hervey Bay Hotel
vallenar - nearest hotel: Hotel del Marqués
kwang binh - nearest hotel: Nhà hàng Amor Palace
salalah - nearest hotel: Muscat International Hotel
al burayqah - nearest hotel: فندق ماس ليبيا
dhahran - nearest hotel: Dhahran International Hotel
khon buri - nearest hotel: The tree
koungou - nearest hotel: Hôtel Trévani
esperanza - nearest hotel: Hotel Novus Caoba
thanatpin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
102,le vauclin,MQ,14.5452,-60.8388,74,Hôtel Macabou
115,fort bragg,US,35.1390,-79.0060,52,Airborne Inn Lodging
133,martapura,ID,-3.4167,114.8500,94,Q-Hotel Grand Dafam
157,point vernon,AU,-25.2500,152.8167,79,Hervey Bay Hotel
175,vallenar,CL,-28.5708,-70.7581,51,Hotel del Marqués
176,kwang binh,VN,17.4833,106.6000,60,Nhà hàng Amor Palace
203,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
231,al burayqah,LY,30.4062,19.5739,51,فندق ماس ليبيا
324,dhahran,SA,26.3032,50.1353,73,Dhahran International Hotel
396,khon buri,TH,14.5254,102.2459,73,The tree


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [90]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng', 'Lat',
                                    geo=True,
                                    tiles='OSM',
                                    frame_width = 650,
                                    frame_height = 400,
                                    color = 'City',
                                    alpha = 0.6,
                                    hover_cols = ['Hotel Name','Country'],
                                    size = 100
                                    )

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)